In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib.animation import FuncAnimation
from IPython import display
from math import sin,pi



#################################################user defined############################################
cfl = 0.1
N = 20
endtime = 2
space_dis = 'cd2' # e or cd2
time_dis = 'cn' #  'cn' or 'rk4' or 'euler'
initials = 's3'# s1 or s3
#################################################End ofuser defined############################################

#################################################PREPROCCES DEFINITIONS############################################
h = 2*pi/N
delt = cfl*h*h
t = np.arange(0,endtime+delt,delt)
x = np.linspace(0,2*pi-h,N)
y = np.linspace(0,2*pi-h,N)
frames = len(t)
S= np.zeros((frames,N, N)) 
x, y = np.meshgrid(x, y)
S[0:,:] = np.sin(x)+np.sin(y)

#################################################END OF PREPROCCES DEFINITIONS############################################

######################################Initial shape conditions###################################
if initials == 's1':
    S[0:,:] = np.sin(x)+np.sin(y)
elif initials == 's3':
    S[0:,:] = np.sin(x)+np.sin(3*y)
    
######################################End of Initial shape conditions###################################


#################################################DERIVATIVES############################################
if space_dis=='cd2' and time_dis=='cn':
    def derivate(N):
        M = np.zeros([N,N])
        for i in range(N-2):        
            M[i,i+1] = 1
            M[i+1,i] = 1
            M[i,i] = -2
        M[0,N-1] = 1
        M[N-1,N-2] = 1
        M[N-1,0] = 1
        M[N-2,N-1] = 1
        M[N-1,N-1] = -2
        M[N-2,N-2] = -2
        M = (1/(h**2))*M
        YM = np.zeros([N**2,N**2])
        for i in range (N**2):
            YM[i,i] = -2
        for i in range(N**2-N):
            YM[i,i+N] = 1
            YM[i+N,i] = 1
        for i in range(N):
            YM[i,i+N**2-N] = 1
            YM[N**2-1-i,N-i-1] = 1
        YM = (1/(h**2))*YM          
        BIGM = np.zeros([N**2,N**2])
        for i in range(N):
            BIGM[i*N:i*N+N,i*N:i*N+N] = M
        ################ DX
#         sx = s.flatten()
#         dx = BIGM.dot(sx)
#         dx = np.reshape(dx, (-1, N))
#         ###################DY
#         sy = s.flatten()
#         dy = YM.dot(sy)
#         dy = np.reshape(dy, (-1, N))
        return BIGM, YM
    
else:
    def derivate(N,s):
        M = np.zeros([N,N])
        for i in range(N-2):        
            M[i,i+1] = 1
            M[i+1,i] = 1
            M[i,i] = -2
        M[0,N-1] = 1
        M[N-1,N-2] = 1
        M[N-1,0] = 1
        M[N-2,N-1] = 1
        M[N-1,N-1] = -2
        M[N-2,N-2] = -2
        M = (1/(h**2))*M
        BIGM = np.zeros([N**2,N**2])
        for i in range(N):
            BIGM[i*N:i*N+N,i*N:i*N+N] = M

        ################ DX
        sx = s.flatten()
        dx = BIGM.dot(sx)
        dx = np.reshape(dx, (-1, N))
        ###################DY
        sy = np.transpose(s)
        sy = sy.flatten()
        dy = BIGM.dot(sy)
        dy = np.reshape(dy, (-1, N))
        dy = np.transpose(dy)
        return dx, dy
#################################################END OF DERIVATIVES############################################





# for i in range(frames-1):
#     S[i+1:,:] = S[i,:,:]-delt*(np.sin(S[i,:,:])+np.sin(S[i,:,:]))

if time_dis == 'euler':
    def solve():      
        for i in range(frames-1):
            S[i+1:,:] = S[i,:,:]+delt*(derivate(N,S[i,:,:])[0]+derivate(N,S[i,:,:])[1])    
        return S
elif time_dis=='rk4':    
    def solve():
        for i in range(frames-1):
            k1x = derivate(N,S[i,:,:])[0]
            k1y = derivate(N,S[i,:,:])[1]
            u2_1 = S[i,:,:] +0.5*delt*(k1x+k1y)
            k2x = derivate(N,u2_1)[0]
            k2y = derivate(N,u2_1)[1]
            u2_2 = S[i,:,:] +0.5*delt*(k2x+k2y)
            k3x = derivate(N,u2_2)[0]
            k3y = derivate(N,u2_2)[1]
            u3 = S[i,:,:] +delt*(k3x+k3y)
            k4x = derivate(N,u3)[0]
            k4y = derivate(N,u3)[1]
            S[i+1,:,:] = S[i,:,:] + (1/6)*delt*(k1x+k1y +2*k2x+2*k2y+2*k3x+2*k3y+k4x+k4y)
        return S
elif time_dis=='cn':
    def  solve():
        I =  np.identity(N**2)
        A1 = I-0.5*delt*derivate(N)[0]
        B1 = I+0.5*delt*derivate(N)[1]
        C1 = I-0.5*delt*derivate(N)[1]
        D1 = I+0.5*delt*derivate(N)[0]
        A1 = np.linalg.inv(A1)
        C1 = np.linalg.inv(C1)
        s = S[0,:,:].flatten()
        for i in range(frames-1):
            dummy = B1.dot(s)
            dummy = A1.dot(dummy)
            dummy = D1.dot(dummy)
            s = C1.dot(dummy)
            S[i+1,:,:] = np.reshape(s, (-1, N))
        return S
    
    
    

S = solve()
def update_plot(frame_number, S, plot):
    plot[0].remove()
    plot[0] = ax.plot_surface(x, y,  S[frame_number,:,:]  , cmap='plasma')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot = [ax.plot_surface(x, y, S[0,:,:], color='1', rstride=1, cstride=1)]
ax.set_zlim(-2,2)
anim = FuncAnimation(fig, update_plot, frames, fargs=(S, plot), interval=10)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()